# 1) Mount drive, unzip data, clone repo, install packages

## 1.1) Mount Drive and define paths
Run provided colab code to mount Google Drive. Then define dataset paths relative to mount point.

In [1]:
!rm -rf /content/sample_data
!rm -rf /content/*.jpg
!rm -rf /content/*.png
!rm -rf /content/*.json

In [2]:
# noinspection PyUnresolvedReferences,PyPackageRequirements
from google.colab import drive
mount_root_abs = '/content/drive'
drive.mount(mount_root_abs)
drive_root = f'{mount_root_abs}/MyDrive'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
# LookBook + (partially) DeepFashion In-shop Clothes Retrieval Benchmark (ICRB)
lb_root_drive = f'{drive_root}/Datasets/LookBook'
assert os.path.exists(lb_root_drive)
lb_img_zip_abs_drive = f'{lb_root_drive}/Img.zip'

# Test if processed Img.zip file exists in dataset root
assert os.path.exists(lb_img_zip_abs_drive), \
  'Please upload a processed zip (processing img.zip in colab will take' + \
  f' for AGES). \nTried: {lb_img_zip_abs_drive}'

## 1.2) Unzip Img directory in Colab
By unzipping the `Img.zip` in Colab before running our model we gain significant disk reading speedups.
So, the first step is to unzip images directory, and then save the image directory before proceeding.

In [4]:
lb_root = lb_root_drive.replace(drive_root, '/content/data')
lb_img_root = f'{lb_root}/Img'
if not os.path.exists(lb_img_root):
    # Clear any previous attempts
    # ATTENTION: This will remove /contents/data/*. So, before running, please 
    # make sure no usable files will be deleted.
    !mkdir -p /content/data
    !rm -rf /content/data

    # Create output directory
    !mkdir -p "$lb_root"

    # Transfer Img.zip from Google Drive to Colab
    lb_img_zip_abs = f'{lb_root}/{os.path.basename(lb_img_zip_abs_drive)}'
    if not os.path.exists(lb_img_zip_abs):
        !cp "$lb_img_zip_abs_drive" "$lb_root"
    # Unzip it in Colab
    !unzip -q "$lb_img_zip_abs" -d "$lb_root"
    # Handle newly-created image directory
    assert os.path.exists(lb_img_root), f'lb_img_root: {lb_img_root}'
    assert not os.path.exists(f'{lb_img_root}/Img')
    !rm -f "$lb_img_zip_abs"
    assert not os.path.exists(lb_img_zip_abs)

    # Create a symbolic link back to drive (we need this to fool GDriveDataset
    # into thinking that it done the unzipping)
    if os.path.exists(f'{lb_root_drive}/Img'):
        !rm "$lb_root_drive"/Img
    !ln -s "$lb_img_root" "$lb_root_drive"

## 1.3) Clone github repo
Clone achariso/gans-thesis repo into /content/code
 using git clone.
 For more info see: https://medium.com/@purba0101/how-to-clone-private-github-repo-in-google-colab-using-ssh-77384cfef18f

In [5]:
repo_root = '/content/code/gans-thesis'
!rm -rf "$repo_root"
if not os.path.exists(repo_root) and not os.path.exists(f'{repo_root}/requirements.txt'):
    # Check that ssh keys exist
    assert os.path.exists(f'{drive_root}/GitHub Keys')
    id_rsa_abs_drive = f'{drive_root}/GitHub Keys/id_rsa'
    id_rsa_pub_abs_drive = f'{id_rsa_abs_drive}.pub'
    assert os.path.exists(id_rsa_abs_drive)
    assert os.path.exists(id_rsa_pub_abs_drive)
    # On first run: Add ssh key in repo
    if not os.path.exists('/root/.ssh'):
        # Transfer config file
        ssh_config_abs_drive = f'{drive_root}/GitHub Keys/config'
        assert os.path.exists(ssh_config_abs_drive)
        !mkdir -p ~/.ssh
        !cp -f "$ssh_config_abs_drive" ~/.ssh/
        # # Add github.com to known hosts
        !ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts
        # Test: !ssh -T git@github.com

    # Remove any previous attempts
    !rm -rf "$repo_root"
    !mkdir -p "$repo_root"
    # Clone repo
    !git clone git@github.com:achariso/gans-thesis.git "$repo_root"
    src_root = f'{repo_root}/src'
    !rm -rf "$repo_root"/report

Cloning into '/content/code/gans-thesis'...
remote: Enumerating objects: 868, done.
remote: Counting objects: 100% (868/868), done.
remote: Compressing objects: 100% (598/598), done.
remote: Total 1479 (delta 593), reused 526 (delta 260), pack-reused 611
Receiving objects: 100% (1479/1479), 791.17 KiB | 1.31 MiB/s, done.
Resolving deltas: 100% (948/948), done.


## 1.4) Install pip packages
All required files are stored in a requirements.txt files at the repository's root.
Use `pip install -r requirements.txt` from inside the dir to install required packages.

In [6]:
%cd "$repo_root"
!pip install -r requirements.txt

/content/code/gans-thesis


In [7]:
import torch
assert torch.cuda.is_available()

## 1.5) Add code/, */src/ to path
This is necessary in order to be able to run the modules.

In [8]:
content_root_abs = f'{repo_root}'
src_root_abs = f'{repo_root}/src'
%env PYTHONPATH="/env/python:$content_root_abs:$src_root_abs

env: PYTHONPATH="/env/python:/content/code/gans-thesis:/content/code/gans-thesis/src


# 2) Train PixelDTGAN model on LookBook + DeepFashion (part of the ICRB dataset)
In this section we run the actual training loop for PixelDTGan network. PixelDTGAN consists of a AE-like generator, and, in our version, two PatchGAN discriminators.

### Colab Bug Workaround
Bug: matplotlib cache not rebuilding.
Solution: Run the following code and then restart the kernel.


In [9]:
# now inside train_pxldtg.py

### Actual Run
Eventually, run the code!

In [10]:
chkpt_step = None       # supported: 'latest', <int>, None
log_level = 'debug'     # supported: 'debug', 'info', 'warning', 'error', 'critical', 'fatal'

# From epoch=37, lambda_recon in G2's loss went from 1 --> 5
# From epoch=66, lambda_recon in G2's loss went from 5 --> 10

# Running with -i enables us to get variables defined inside the script (the script runs inline)
%run -i src/train_setup.py --log_level $log_level --chkpt_step $chkpt_step --seed 42


### PixelDTGAN Training


In [11]:
%cd src/

import torch
from IPython.core.display import display
from torch import Tensor
from torch.nn import DataParallel
# noinspection PyProtectedMember
from torch.utils.data import DataLoader

from datasets.look_book import PixelDTDataset, PixelDTDataloader
from modules.pixel_dt_gan import PixelDTGan
from utils.dep_free import get_tqdm
from utils.ifaces import FilesystemDataset
from utils.metrics import GanEvaluator

###################################
###  Hyper-parameters settings  ###
###################################
# TODO: finish this notebook and train in Colab/Kaggle
#   - training
n_epochs = 100
batch_size = 48 if not run_locally else 48
train_test_splits = [90, 10]  # for a 90% training - 10% evaluation set split
#   - evaluation
metrics_n_samples = 1000 if not run_locally else 2
metrics_batch_size = 32 if not run_locally else 1
f1_k = 3 if not run_locally else 1
#   - visualizations / checkpoints steps
display_step = 200
checkpoint_step = 600
metrics_step = 1800  # evaluate model every 3 checkpoints
#   - dataset
target_shape = 128
target_channels = 3
#   - PixelDTGAN configiguration
pxldtg_config_id = f'default'  # as proposed in the original paper

###################################
###   Dataset Initialization    ###
###################################
#   - image transforms:
#     If target_shape is different from load one, resize & crop. If target_shape is different from load shape,
#     convert to grayscale.
#     Update: Now done automatically if you set target_channels, target_shape when instantiating the dataloader.
gen_transforms = PixelDTDataset.get_image_transforms(target_shape=PixelDTGan.DefaultConfiguration['shapes']['w_in'], target_channels=3)
#   - the dataloader used to access the training dataset of cross-scale/pose image pairs at every epoch
#     > len(dataloader) = <number of batches>
#     > len(dataloader.dataset) = <number of total dataset items>
dataloader = PixelDTDataloader(dataset_fs_folder_or_root=datasets_groot, batch_size=batch_size,
                               image_transforms=gen_transforms, splits=train_test_splits,
                               pin_memory=not run_locally, log_level=log_level)
dataset = dataloader.dataset
#   - ensure dataset is fetched locally and unzipped
if isinstance(dataset, FilesystemDataset):
    dataset.fetch_and_unzip(in_parallel=False, show_progress=True)
elif hasattr(dataset, 'dataset') and isinstance(dataset.dataset, FilesystemDataset):
    dataset.dataset.fetch_and_unzip(in_parallel=False, show_progress=True)
else:
    raise TypeError('dataset must implement utils.ifaces.FilesystemDataset in order to be auto-downloaded and unzipped')
#   - apply rudimentary tests
assert issubclass(dataloader.__class__, DataLoader)
assert len(dataloader) == len(dataset) // batch_size + (1 if len(dataset) % batch_size else 0)
_img_s, _img_t = next(iter(dataloader))
assert tuple(_img_s.shape) == (batch_size, target_channels, target_shape, target_shape)
assert tuple(_img_t.shape) == (batch_size, target_channels, target_shape, target_shape)

###################################
###    Models Initialization    ###
###################################
#   - initialize evaluator instance (used to run GAN evaluation metrics: FID, IS, PRECISION, RECALL, F1 and SSIM)
evaluator = GanEvaluator(model_fs_folder_or_root=models_groot, gen_dataset=dataset, target_index=1, device=exec_device,
                         condition_indices=(0, ), n_samples=metrics_n_samples, batch_size=metrics_batch_size,
                         f1_k=f1_k)
#   - initialize model
chkpt_step = args.chkpt_step
try:
    if chkpt_step == 'latest':
        pxldtg_chkpt_step = chkpt_step
    elif isinstance(chkpt_step, str) and chkpt_step.isdigit():
        pxldtg_chkpt_step = int(chkpt_step)
    else:
        pxldtg_chkpt_step = None
except NameError:
    pxldtg_chkpt_step = None
pxldtg = PixelDTGan(model_fs_folder_or_root=models_groot, config_id=pxldtg_config_id, dataset_len=len(dataset),
            chkpt_epoch=pxldtg_chkpt_step, evaluator=evaluator, device=exec_device, log_level=log_level)
pxldtg.logger.debug(f'Using device: {str(exec_device)}')
pxldtg.logger.debug(f'Model initialized. Number of params = {pxldtg.nparams_hr}')
#   - setup multi-GPU training
if torch.cuda.device_count() > 1:
    pxldtg.gen = DataParallel(pxldtg.gen)
    pxldtg.info(f'Using {torch.cuda.device_count()} GPUs for PixelDTGAN Generator (via torch.nn.DataParallel)')
#   - load dataloader state (from model checkpoint)
if 'dataloader' in pxldtg.other_state_dicts.keys():
    dataloader.set_state(pxldtg.other_state_dicts['dataloader'])
    pxldtg.logger.debug(f'Loaded dataloader state! Current pem_index={dataloader.get_state()["perm_index"]}')


/content/code/gans-thesis/src


  DEBUG    | Found 71.7K image pairs in dataset
  DEBUG    | Split dataset: len(training_set)=64.6K, len(test_set)=7.2K
  DEBUG    | Dataset is fetched and unzipped!
  DEBUG    | Using device: cuda:0
  DEBUG    | Model initialized. Number of params = 59.9M


### PixelDTGAN Main training loop


In [ ]:
import click
torch.cuda.empty_cache()

###################################
###       Training Loop         ###
###################################
#   - get the correct tqdm instance
exec_tqdm = get_tqdm()
#   - start training loop from last checkpoint's epoch and step
gcapture_ready = True
async_results = None
pxldtg.logger.info(f'[training loop] STARTING (epoch={pxldtg.epoch}, step={pxldtg.initial_step})')
for epoch in range(pxldtg.epoch, n_epochs):
    image_1: Tensor
    image_2: Tensor
    pose_2: Tensor

    # noinspection PyProtectedMember
    d = {
        'step': pxldtg.step,
        'initial_step': pxldtg.initial_step,
        'epoch': pxldtg.epoch,
        '_counter': pxldtg._counter,
        'epoch_inc': pxldtg.epoch_inc,
    }
    # initial_step = pxldtg.initial_step % len(dataloader)
    pxldtg.logger.debug('[START OF EPOCH] ' + str(d))
    for img_s, img_t in get_tqdm()(dataloader, initial=pxldtg.initial_step):
        # Transfer image batches to GPU
        img_s = img_s.to(exec_device)
        img_t = img_t.to(exec_device)

        # Perform a forward + backward pass + weight update on the Generator & Discriminator models
        disc_r_loss, disc_a_loss, gen_loss = pxldtg(img_s, img_t)

        # Metrics & Checkpoint Code
        if pxldtg.step % checkpoint_step == 0:
            # Check if another upload is pending
            if not gcapture_ready and async_results:
                # Wait for previous upload to finish
                pxldtg.logger.warning('Waiting for previous gcapture() to finish...')
                [r.wait() for r in async_results]
                pxldtg.logger.warning('DONE! Starting new capture now.')
            # Capture current model state, including metrics and visualizations
            async_results = pxldtg.gcapture(checkpoint=True, metrics=pxldtg.step % metrics_step == 0, visualizations=True,
                                          dataloader=dataloader, in_parallel=True, show_progress=True,
                                          delete_after=False)
        # Visualization code
        elif pxldtg.step % display_step == 0:
            visualization_img = pxldtg.visualize()
            visualization_img.show() if not in_notebook() else display(visualization_img)

        # Check if a pending checkpoint upload has finished
        if async_results:
            gcapture_ready = all([r.ready() for r in async_results])
            if gcapture_ready:
                pxldtg.logger.info(f'gcapture() finished')
                if pxldtg.latest_checkpoint_had_metrics:
                    pxldtg.logger.info(str(pxldtg.latest_metrics))
                async_results = None

        # If run locally one pass is enough
        if run_locally and gcapture_ready:
            break

    # If run locally one pass is enough
    if run_locally:
        break

    # noinspection PyProtectedMember
    d = {
        'step': pxldtg.step,
        'initial_step': pxldtg.initial_step,
        'epoch': pxldtg.epoch,
        '_counter': pxldtg._counter,
        'epoch_inc': pxldtg.epoch_inc,
    }
    pxldtg.logger.debug('[END OF EPOCH] ' + str(d))

# Check if a pending checkpoint exists
if async_results:
    ([r.wait() for r in async_results])
    pxldtg.logger.info(f'last gcapture() finished')
    if pxldtg.latest_checkpoint_had_metrics:
        pxldtg.logger.info(str(pxldtg.latest_metrics))
    async_results = None

# Training finished!
pxldtg.logger.info('[training loop] DONE')


  INFO     | [training loop] STARTING (epoch=0, step=0)
  DEBUG    | [START OF EPOCH] {'step': None, 'initial_step': 0, 'epoch': 0, '_counter': 0, 'epoch_inc': False}


# 3) Evaluate PixelDTGAN
In this section we evaluate the generation performance of our trained network using the SOTA GAN evaluation metrics.

## 3.1) Get the metrics evolution plots
We plot how the metrics evolved during training. The GAN is **not** trained to minimize those metrics (they are
calculated using `torch.no_grad()`) and thus this evolution merely depends on the network and showcases the correlation
between the GAN evaluation metrics, and the losses (e.g. adversarial & reconstruction) used to optimize the network.

In [ ]:
# Since the PixelDTGAN implements utils.ifaces.Visualizable, we can
# directly call visualize_metrics() on the model instance.
_ = pxldtg.visualize_metrics(upload=True, preview=True)

## 3.2) Evaluate Generated Samples
In order to evaluate generated samples and compare model with other GAN architectures trained on the same dataset. For this purpose we will re-calculate the evaluation metrics as stated above, but with a much bigger number of samples. In this way, the metrics will be more trustworthy and comparable with the corresponding metrics in the original paper.


In [ ]:
# Initialize a new evaluator instance
# (used to run GAN evaluation metrics: FID, IS, PRECISION, RECALL, F1 and SSIM)
evaluator = GanEvaluator(model_fs_folder_or_root=models_groot, gen_dataset=dataset, target_index=1, device=exec_device,
                         condition_indices=(0, 2), n_samples=10000, batch_size=metrics_batch_size,
                         f1_k=f1_k, ssim_c_img=target_channels)
# Run the evaluator
metrics_dict = evaluator.evaluate(gen=pxldtg.gen, metric_name='all', show_progress=True)

# Print results
import json
print(json.dumps(metrics_dict, indent=4))

#-----------
# Epoch 93
#----------
# {
#   "fid": 16.195581436157227
#   "is": 2.82967472076416
#   "f1": 0.8827780485153198
#   "precision": 0.8856828808784485
#   "recall": 0.8798921704292297
#   "ssim": 0.8029271364212036
# }